In [1]:
import pandas as pd  
df = pd.read_json('wiki.json')

In [2]:
df

,language,image_url,page_title,caption_reference_description,caption_attribution_description,mime_type
0,en,https://upload.wikimedia.org/wikipedia/commons...,Oxydactylus,None,English: Mounted skeleton of Oxydactylus longi...,image/jpeg
1,en,https://upload.wikimedia.org/wikipedia/commons...,Maine's 3rd congressional district,None,"English: Samuel W. Gould, US Representative fr...",image/jpeg
2,en,https://upload.wikimedia.org/wikipedia/commons...,Cheraw,A c. 1724 English copy of a deerskin Catawba m...,"English: ""Map of the Several Nations of Indian...",image/jpeg
3,en,https://upload.wikimedia.org/wikipedia/commons...,Romanian Front,"Nameplate of Gazeta Transilvaniei on June 14, ...",English: Nameplate of the Romanian nationalist...,image/png
4,en,https://upload.wikimedia.org/wikipedia/commons...,Ádám Kósa,None,English: Hungarian MEP Ádám Kósa,image/jpeg
...,...,...,...,...,...,...
5411973,en,https://upload.wikimedia.org/wikipedia/commons...,List of vice presidents of the Examination Yuan,None,"English: Jia Jingde, politician of China. 中文: ...",image/jpeg
5411974,en,https://upload.wikimedia.org/wikipedia/commons...,Brennen Beyer,Beyer in 2013,DSC_9909.jpg,image/jpeg
5411975,en,https://upload.wikimedia.org/wikipedia/commons...,Kannagi,Kannagi in Tamil Nadu.,"English: Idol of Kannaki Amman at Sanctum, Tha...",image/jpeg
5411976,en,https://upload.wikimedia.org/wikipedia/commons...,Don Det,None,English: Landscape with stormy clouds and a pi...,image/jpeg


In [3]:
df_en = df[df['caption_reference_description'].notna()]

In [4]:
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install transformers>=4.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from torchmetrics.multimodal.clip_score import CLIPScore
metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

In [6]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install torchvision

  Using cached torchvision-0.14.1-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import urllib
import numpy as np
import urllib.request
import torchvision.transforms as transforms

def load_img_from_url(image_url):
  try:
    response = urllib.request.urlopen(image_url)
    arr = np.asarray(bytearray(response.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, -1)
    if cv2.waitKey() & 0xff == 27:
      return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    transform = transforms.Compose([
      transforms.ToTensor()
    ])
    tensor = transform(img)
    return tensor
  except:
    return None

In [6]:
from tqdm import tqdm

In [7]:
scores = []
df_clip = df_en.iloc[0:20]
for index, row in tqdm(df_clip.iterrows(), total=df_clip.shape[0]):
  image = load_img_from_url(row['image_url'])
  if image is not None:
    scores.append(metric(image, row['caption_reference_description']))

100%|█████████████████████████████████████████████████████████████████| 20/20 [00:56<00:00,  2.83s/it]


In [8]:
scores

[tensor(20.7068, grad_fn=<SqueezeBackward0>),
 tensor(26.6524, grad_fn=<SqueezeBackward0>),
 tensor(24.7295, grad_fn=<SqueezeBackward0>),
 tensor(21.0817, grad_fn=<SqueezeBackward0>),
 tensor(22.7854, grad_fn=<SqueezeBackward0>),
 tensor(20.8798, grad_fn=<SqueezeBackward0>),
 tensor(22.9363, grad_fn=<SqueezeBackward0>),
 tensor(22.6360, grad_fn=<SqueezeBackward0>),
 tensor(22.0836, grad_fn=<SqueezeBackward0>),
 tensor(21.2938, grad_fn=<SqueezeBackward0>),
 tensor(22.1035, grad_fn=<SqueezeBackward0>),
 tensor(21.4331, grad_fn=<SqueezeBackward0>),
 tensor(23.3386, grad_fn=<SqueezeBackward0>),
 tensor(18.9644, grad_fn=<SqueezeBackward0>),
 tensor(26.2554, grad_fn=<SqueezeBackward0>),
 tensor(22.8616, grad_fn=<SqueezeBackward0>),
 tensor(20.2094, grad_fn=<SqueezeBackward0>),
 tensor(20.7328, grad_fn=<SqueezeBackward0>),
 tensor(22.4470, grad_fn=<SqueezeBackward0>)]

In [12]:
%pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [10]:
# fig_, ax_ = metric.plot(scores)

In [12]:
import re
R = r'\b(?![A-Za-z0-9]+[!"#$%&\'()*+,\\ -./:;<=>?@[\]^{|}])\w+\b'
df_en.loc[:, 'caption_reference_description'] = df_en.caption_reference_description.apply(lambda x : np.nan if pd.isna(x) else re.sub(R, '', (str(x) + ' ')))

In [13]:
mask = ((df_en.caption_reference_description == '') | (df_en.caption_reference_description.str.contains('^[0-9!"#$%&\'()*+,\\ -./:;<=>?@[\]^{|}\s]+$')))
df_en = df_en[~mask]

In [14]:
R = re.compile(r'[A-Za-z0-9]')
df_en.loc[:, 'caption_reference_description'] = df_en.caption_reference_description.apply(lambda x: str(x)[: (0 if len(R.findall(str(x))) == 0 else str(x).rfind(R.findall(str(x))[-1])) + 1])

In [16]:
df_en = df_en.dropna(subset=['caption_reference_description', 'image_url'])

In [17]:
df_en = df_en.drop_duplicates(subset='image_url')
df_en

,language,image_url,page_title,caption_reference_description,caption_attribution_description,mime_type
2,en,https://upload.wikimedia.org/wikipedia/commons...,Cheraw,A c. 1724 English copy of a deerskin Catawba m...,"English: ""Map of the Several Nations of Indian...",image/jpeg
3,en,https://upload.wikimedia.org/wikipedia/commons...,Romanian Front,"Nameplate of Gazeta Transilvaniei on June 14, ...",English: Nameplate of the Romanian nationalist...,image/png
5,en,https://upload.wikimedia.org/wikipedia/commons...,Grey francolin,A captive decoy,English: A decoy grey francolin used by a trap...,image/jpeg
10,en,https://upload.wikimedia.org/wikipedia/commons...,U.S. Route 41 in Michigan,The former Bus. 41 along Washington Street in...,from left to right: the Old State Savings Bank...,image/jpeg
12,en,https://upload.wikimedia.org/wikipedia/commons...,Skolkovo Innovation Center,"Viktor Vekselberg, head of the Skolkovo Instit...",English: “Presentation by Skolkovo Institute o...,image/jpeg
...,...,...,...,...,...,...
5411960,en,https://upload.wikimedia.org/wikipedia/commons...,Urban Transportation Development Corporation,Massachusetts Bay Transportation Authority 170...,English: UTDC/Bombardier 1700 series Red Line ...,image/jpeg
5411961,en,https://upload.wikimedia.org/wikipedia/commons...,"Rocquigny, Ardennes",Fortified church,Français&#160;: Rocquigny (Ardennes) Église fo...,image/jpeg
5411966,en,https://upload.wikimedia.org/wikipedia/commons...,Bisaldeo temple,Bisaldeo temple in 2016 with submerged courtyard,English: this temple is submerged in water aft...,image/jpeg
5411974,en,https://upload.wikimedia.org/wikipedia/commons...,Brennen Beyer,Beyer in 2013,DSC_9909.jpg,image/jpeg


In [18]:
df_en = df_en.loc[df["caption_reference_description"].str.count(" ") <= 20, :]

In [19]:
df_en.index = list(range(df_en.shape[0]))

In [20]:
df_en

,language,image_url,page_title,caption_reference_description,caption_attribution_description,mime_type
0,en,https://upload.wikimedia.org/wikipedia/commons...,Romanian Front,"Nameplate of Gazeta Transilvaniei on June 14, ...",English: Nameplate of the Romanian nationalist...,image/png
1,en,https://upload.wikimedia.org/wikipedia/commons...,Grey francolin,A captive decoy,English: A decoy grey francolin used by a trap...,image/jpeg
2,en,https://upload.wikimedia.org/wikipedia/commons...,U.S. Route 41 in Michigan,The former Bus. 41 along Washington Street in...,from left to right: the Old State Savings Bank...,image/jpeg
3,en,https://upload.wikimedia.org/wikipedia/commons...,Skolkovo Innovation Center,"Viktor Vekselberg, head of the Skolkovo Instit...",English: “Presentation by Skolkovo Institute o...,image/jpeg
4,en,https://upload.wikimedia.org/wikipedia/commons...,Blood Swept Lands and Seas of Red,Volunteer planting poppies,Tower of London Remembers. To mark the centena...,image/jpeg
...,...,...,...,...,...,...
2349693,en,https://upload.wikimedia.org/wikipedia/commons...,Urban Transportation Development Corporation,Massachusetts Bay Transportation Authority 170...,English: UTDC/Bombardier 1700 series Red Line ...,image/jpeg
2349694,en,https://upload.wikimedia.org/wikipedia/commons...,"Rocquigny, Ardennes",Fortified church,Français&#160;: Rocquigny (Ardennes) Église fo...,image/jpeg
2349695,en,https://upload.wikimedia.org/wikipedia/commons...,Bisaldeo temple,Bisaldeo temple in 2016 with submerged courtyard,English: this temple is submerged in water aft...,image/jpeg
2349696,en,https://upload.wikimedia.org/wikipedia/commons...,Brennen Beyer,Beyer in 2013,DSC_9909.jpg,image/jpeg


In [21]:
from torchmetrics.multimodal.clip_score import CLIPScore
metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

In [22]:
scores = []
df_clip = df_en.iloc[0:20]
for index, row in tqdm(df_clip.iterrows(), total=df_clip.shape[0]):
  image = load_img_from_url(row['image_url'])
  if image is not None:
    scores.append(metric(image, row['caption_reference_description']))

100%|█████████████████████████████████████████████████████████████████| 20/20 [01:03<00:00,  3.17s/it]


In [23]:
scores

[tensor(27.5785, grad_fn=<SqueezeBackward0>),
 tensor(24.7295, grad_fn=<SqueezeBackward0>),
 tensor(20.7927, grad_fn=<SqueezeBackward0>),
 tensor(22.7854, grad_fn=<SqueezeBackward0>),
 tensor(20.8798, grad_fn=<SqueezeBackward0>),
 tensor(22.9363, grad_fn=<SqueezeBackward0>),
 tensor(23.7782, grad_fn=<SqueezeBackward0>),
 tensor(21.2938, grad_fn=<SqueezeBackward0>),
 tensor(22.1035, grad_fn=<SqueezeBackward0>),
 tensor(21.5909, grad_fn=<SqueezeBackward0>),
 tensor(23.3386, grad_fn=<SqueezeBackward0>),
 tensor(18.9644, grad_fn=<SqueezeBackward0>),
 tensor(26.2554, grad_fn=<SqueezeBackward0>),
 tensor(23.1314, grad_fn=<SqueezeBackward0>),
 tensor(20.2094, grad_fn=<SqueezeBackward0>),
 tensor(20.7328, grad_fn=<SqueezeBackward0>),
 tensor(22.4470, grad_fn=<SqueezeBackward0>),
 tensor(21.2370, grad_fn=<SqueezeBackward0>),
 tensor(23.7377, grad_fn=<SqueezeBackward0>)]

In [25]:
# fig_, ax_ = metric.plot(scores)

In [26]:
df_en = df_en[['image_url', 'caption_reference_description']]

In [27]:
df_en = df_en.rename(columns={'caption_reference_description' : 'caption_description'})

In [28]:
json = df_en.to_json(orient="records")

In [30]:
with open("wiki_en.json", "w") as outfile: 
    outfile.write(json)

Датасет на русском языке

In [31]:
df_r = pd.read_json('wiki_ru_source.json')

In [32]:
df_ru = df_r[df_r['caption_reference_description'].notna()]

In [35]:
import re
R = r'\b(?![а-яА-ЯёЁ0-9]+[!"#$%&\'()*+,\\ -./:;<=>?@[\]^{|}])\w+\b'
df_ru.loc[:, 'caption_reference_description'] = df_ru.caption_reference_description.apply(lambda x : np.nan if pd.isna(x) else re.sub(R, '', (str(x) + ' ')))

In [36]:
mask_ru = ((df_ru.caption_reference_description == '') | (df_ru.caption_reference_description.str.contains('^[0-9!"#$%&\'()*+,\\ -./:;<=>?@[\]^{|}\s]+$')))
df_ru = df_ru[~mask_ru]

In [37]:
R = re.compile(r'[а-яА-ЯёЁ0-9]')
df_ru.loc[:, 'caption_reference_description'] = df_ru.caption_reference_description.apply(lambda x: str(x)[: (0 if len(R.findall(str(x))) == 0 else str(x).rfind(R.findall(str(x))[-1])) + 1])

In [38]:
df_ru = df_ru.dropna(subset=['caption_reference_description', 'image_url'])

In [39]:
df_ru = df_ru.drop_duplicates(subset='image_url')

In [40]:
df_ru = df_ru.loc[df_ru["caption_reference_description"].str.count(" ") <= 20, :]

In [41]:
df_ru.index = list(range(df_ru.shape[0]))

In [42]:
df_ru

,language,image_url,page_title,caption_reference_description,caption_attribution_description,mime_type
0,ru,https://upload.wikimedia.org/wikipedia/commons...,Умершие в ноябре 2013 года,Владислав Захаревич,Русский: Ректор ЮФУ В.Г. ЗахаревичEnglish: Pro...,image/jpeg
1,ru,https://upload.wikimedia.org/wikipedia/commons...,Буш (курган),Зарисовка крупнейшего из Золотых ромбов курган...,English: design of the the Bush Barrow Lozenge...,image/jpeg
2,ru,https://upload.wikimedia.org/wikipedia/commons...,Трассовый моделизм,Трассовый моделизм. Прохождение моделями виража,日本語: YSR race Mr.ky,image/jpeg
3,ru,https://upload.wikimedia.org/wikipedia/commons...,Лилейная трещалка,Спаривание,"Liloceris lilii (Scopoli, 1763), copula Englis...",image/jpeg
4,ru,https://upload.wikimedia.org/wikipedia/commons...,"Бурылин, Дмитрий Геннадьевич",Музей Дмитрия Бурылина,English: Museum of Industry and Art (Burylin M...,image/jpeg
...,...,...,...,...,...,...
449907,ru,https://upload.wikimedia.org/wikipedia/commons...,Мужчина,Изображение мужской особи человека (самец вида...,English: The svg version isn't an exact replic...,image/png
449908,ru,https://upload.wikimedia.org/wikipedia/commons...,Пасхальный цыплёнок,Пасхальное украшение. ЖД станция в Норрчёпинге...,Svenska: Påsken 2013 vid Norrköpings järnvägst...,image/jpeg
449909,ru,https://upload.wikimedia.org/wikipedia/commons...,Селезениха (Кировская область),Вид на «Красный ». Осень 2017 г,Русский: Лето 2018 г.,image/jpeg
449910,ru,http://upload.wikimedia.org/wikipedia/commons/...,Императорские и королевские уланы,Максимилиан фон Родаковский во главе 13-го пол...,German: Oberst Maximilian Rodakowski an der Sp...,image/jpeg


In [43]:
df_ru = df_ru[['image_url', 'caption_reference_description']]

In [44]:
df_ru = df_ru.rename(columns={'caption_reference_description' : 'caption_description'})

In [45]:
json_ru = df_ru.to_json(orient="records")

In [46]:
with open("wiki_ru.json", "w") as outfile: 
    outfile.write(json_ru)